# Importations et Configuration Initiale

In [ ]:
import enoslib as en
import logging
import os
from pathlib import Path
import time

# --- Configuration de l'expérience ---

# Configurez le logging pour avoir un retour clair sur les étapes
logging.basicConfig(level=logging.INFO)



G5K_SITE = "rennes"  # Site Grid'5000 à utiliser

# Nom du job pour la réservation sur Grid'5000
JOB_NAME = "K8sEnoslibDeploy"

# Clé SSH, par défaut enoslib cherche ~/.ssh/id_rsa
# Si vous en utilisez une autre, décommentez et modifiez la ligne suivante :
# en.set_config(ssh_key="~/.ssh/ma_cle_ssh")

# Création du chemin pour les sorties (par exemple, inventaire)
Path("inventory").mkdir(exist_ok=True)
OUTPUT_INVENTORY = "inventory/g5k_inventory.yaml"

print("Configuration initiale chargée.")


# Définition des Ressources pour le Cluster

In [ ]:
# CELLULE 2
# Définition de la configuration pour la connexion à Grid'5000
# en.set_provider(
#     en.G5k(
#         username=G5K_USER,
#         site=G5K_SITE,
#         job_name=JOB_NAME,
#     )
# )

# Définition des ressources à réserver
# 1 nœud master et 2 nœuds workers
# Tous les nœuds seront déployés avec Ubuntu 22.04
conf = (
en.G5kConf.from_settings(
    job_name=JOB_NAME,
    job_type=["deploy"],
    walltime="03:30:00", # Réservez pour 1 heure
    env_name="ubuntu2204-min"
).add_machine(
    roles=["master"],
    cluster="paradoxe",
    nodes=1
).add_machine(
    roles=["workers"],
    cluster="paradoxe",
    nodes=2
)
)

print("Définition des ressources terminée.")

# Exécution de la Réservation et du Déploiement

In [ ]:

provider = en.G5k(conf)
# Démarrage de la réservation et du déploiement
# provider.init() bloque jusqu'à ce que les noeuds soient prêts

roles, networks = provider.init()

# Récupération des informations sur les rôles pour les cellules suivantes
#roles = reservation.get_roles()
master_node = roles["master"]
worker_nodes = roles["workers"]

print("--- Réservation et déploiement terminés ! ---")
print(f"Master: {[node.address for node in master_node]}")
print(f"Workers: {[node.address for node in worker_nodes]}")
#print(f"Tous les nœuds: roles" + str(roles))

#  Préparation Commune de Tous les Nœuds

In [ ]:
# Script de configuration commun
COMMON_SETUP_SCRIPT = """
#!/bin/bash -xe
sudo apt-get update -y
sudo apt-get install -y software-properties-common gpg curl apt-transport-https ca-certificates jq
cat <<EOF | sudo tee /etc/modules-load.d/k8s.conf
overlay
br_netfilter
EOF
sudo modprobe overlay
sudo modprobe br_netfilter
cat <<EOF | sudo tee /etc/sysctl.d/k8s.conf
net.bridge.bridge-nf-call-iptables  = 1
net.bridge.bridge-nf-call-ip6tables = 1
net.ipv4.ip_forward                 = 1
EOF
sudo sysctl --system
sudo swapoff -a
curl -fsSL https://pkgs.k8s.io/addons:/cri-o:/prerelease:/main/deb/Release.key | gpg --dearmor | sudo tee /etc/apt/keyrings/cri-o-apt-keyring.gpg >/dev/null
echo "deb [signed-by=/etc/apt/keyrings/cri-o-apt-keyring.gpg] https://pkgs.k8s.io/addons:/cri-o:/prerelease:/main/deb/ /" | sudo tee /etc/apt/sources.list.d/cri-o.list
sudo apt-get update -y
sudo apt-get install -y cri-o
sudo systemctl enable --now crio
KUBERNETES_VERSION="1.30"
sudo mkdir -p /etc/apt/keyrings
curl -fsSL https://pkgs.k8s.io/core:/stable:/v$KUBERNETES_VERSION/deb/Release.key | sudo gpg --dearmor -o /etc/apt/keyrings/kubernetes-apt-keyring.gpg
echo "deb [signed-by=/etc/apt/keyrings/kubernetes-apt-keyring.gpg] https://pkgs.k8s.io/core:/stable:/v$KUBERNETES_VERSION/deb/ /" | sudo tee /etc/apt/sources.list.d/kubernetes.list
sudo apt-get update
sudo apt-get install -y kubelet=1.30.0-1.1 kubectl=1.30.0-1.1 kubeadm=1.30.0-1.1
sudo apt-mark hold kubelet kubeadm kubectl
IFACE=$(ip route | grep default | awk '{print $5}')
local_ip=$(ip --json addr show $IFACE | jq -r '.[0].addr_info[] | select(.family == "inet") | .local')
echo "KUBELET_EXTRA_ARGS=--node-ip=$local_ip" | sudo tee /etc/default/kubelet
"""

# Exécution du script sur tous les nœuds
print("Lancement de la préparation commune...")
with en.actions(roles=roles) as p:
    p.shell(COMMON_SETUP_SCRIPT)
print("Préparation commune terminée.")

##  Vérification de la Préparation des Nœuds

In [ ]:
# Script de vérification (inchangé)
VERIFY_PREP_SCRIPT = """
#!/bin/bash -e
echo "--- Vérification de l'état des nœuds ---"
echo "[VÉRIF] Service CRI-O (runtime de conteneurs)..."
sudo systemctl is-active --quiet crio
echo "✅ OK: Le service CRI-O est actif."
echo "[VÉRIF] Binaires Kubernetes..."
kubelet --version
kubeadm version
echo "✅ OK: Les binaires Kubelet et Kubeadm sont installés."
echo "[VÉRIF] Modules Kernel..."
lsmod | grep -q br_netfilter && echo "✅ OK: Module 'br_netfilter' est chargé."
lsmod | grep -q overlay && echo "✅ OK: Module 'overlay' est chargé."
echo "\n🎉 La préparation de ce nœud est validée."
"""

# 1. On exécute les actions (comme avant)
print("Lancement de la vérification sur tous les nœuds...")
with en.actions(roles=roles) as p:
    p.shell(VERIFY_PREP_SCRIPT)

results = p.results
verify = results[0].stdout.strip()
print(verify)
# # 2. ✅ NOUVEAU : On récupère et on affiche les résultats APRÈS le bloc
# print("\n--- RÉSULTATS DE LA VÉRIFICATION ---")
# results = p.get_results()

# # On boucle sur les résultats de chaque machine
# for r in results:
#     print(f"\n======== Nœud: {r.host} ========")
#     print(r.stdout) # Affiche la sortie standard de la commande
    
#     # Optionnel : Affiche les erreurs s'il y en a
#     if r.stderr:
#         print(f"-------- Erreurs (stderr) pour {r.host} --------")
#         print(r.stderr)

#  Initialisation du Master

In [ ]:
# Script d'initialisation du master
MASTER_INIT_SCRIPT = """
#!/bin/bash -xe
IFACE=$(ip route | grep default | awk '{print $5}')
IPADDR=$(ip -4 addr show $IFACE | grep -oP '(?<=inet\s)\d+(\.\d+){3}')
NODENAME=$(hostname -s)
sudo kubeadm init --pod-network-cidr=192.168.0.0/16 --apiserver-advertise-address=$IPADDR --node-name $NODENAME --cri-socket unix:///var/run/crio/crio.sock --ignore-preflight-errors=Swap
mkdir -p $HOME/.kube
sudo cp -i /etc/kubernetes/admin.conf $HOME/.kube/config
sudo chown $(id -u):$(id -g) $HOME/.kube/config
kubectl apply -f https://docs.projectcalico.org/manifests/calico.yaml
"""

# Exécution du script sur le master
print("Initialisation du master...")
with en.actions(roles=master_node) as p:
    p.shell(MASTER_INIT_SCRIPT)
print("Initialisation du master terminée.")

# Jonction des Workers

In [ ]:
# 1. Récupérer la commande de jonction depuis le master
print("Récupération de la commande de jonction depuis le master...")
with en.actions(roles=master_node) as p:
    p.shell("sudo kubeadm token create --print-join-command")

# Récupération des résultats après le bloc
results = p.results
join_command = results[0].stdout.strip()
print(f"✅ Commande de jonction récupérée: {join_command}")

# 2. Exécuter la commande sur les workers
print("Les workers rejoignent le cluster...")
with en.actions(roles=worker_nodes) as p:
    p.shell(f"sudo {join_command}")

print("Workers joints. Attente de 60s pour la stabilisation...")
#time.sleep(60)

# Vérification du Cluster

In [ ]:
# Exécution de kubectl get nodes sur le master pour vérifier
with en.actions(roles=master_node) as p:
    p.shell("kubectl get nodes -o wide")

results = p.results
nodes = results[0].stdout.strip()
print("--- État du cluster ---")
print(nodes)

 ## Installation des Add-ons

In [ ]:
# Script pour installer les add-ons
ADDONS_SCRIPT = """
#!/bin/bash -e

echo "--- Installation du Metrics Server ---"
kubectl apply -f https://raw.githubusercontent.com/techiescamp/kubeadm-scripts/main/manifests/metrics-server.yaml

echo ""
echo "--- Installation d'Ingress-NGINX ---"
kubectl apply -f https://raw.githubusercontent.com/kubernetes/ingress-nginx/controller-v1.13.0/deploy/static/provider/baremetal/deploy.yaml

echo ""
echo "Installation des add-ons terminée."
"""

# Exécution du script sur le master
print("Installation des add-ons sur le nœud master...")
with en.actions(roles=master_node) as p:
    p.shell(ADDONS_SCRIPT)

# Récupération de la sortie comme vous l'avez spécifié
print("\n--- Résultat de l'installation ---")
if p.results:
    output = p.results[0].stdout.strip()
    print(output)
else:
    print("Aucun résultat à afficher. Vérifiez les erreurs potentielles.")

# Déploiement et Test d'une Application

In [ ]:
# Script pour déployer l'application de test et son service
APP_DEPLOY_SCRIPT = """
#!/bin/bash -e

echo "--- Création du Deployment et du Service pour l'application echoserver ---"

# On utilise un 'heredoc' pour passer le YAML directement à kubectl
cat <<EOF | kubectl apply -f -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: echoserver
spec:
  replicas: 2
  selector:
    matchLabels:
      app: echoserver
  template:
    metadata:
      labels:
        app: echoserver
    spec:
      containers:
      - name: echoserver
        image: registry.k8s.io/echoserver:1.10
        ports:
        - containerPort: 8080
---
apiVersion: v1
kind: Service
metadata:
  name: echoserver
spec:
  type: NodePort
  selector:
    app: echoserver
  ports:
  - port: 80
    targetPort: 8080
EOF

echo "\n--- Attente de 15 secondes pour le démarrage des pods ---"
sleep 15

echo "\n--- Vérification du statut du déploiement ---"
kubectl get deployment echoserver

echo "\n--- Vérification des pods (devrait en afficher 2) ---"
kubectl get pods -l app=echoserver

echo "\n--- Vérification du service (notez le port après '80:') ---"
kubectl get service echoserver
"""

# Exécution du script sur le master
print("Déploiement de l'application de test...")
with en.actions(roles=master_node) as p:
    p.shell(APP_DEPLOY_SCRIPT)

# Affichage du résultat
print("\n--- Résultat du déploiement ---")
if p.results:
    output = p.results[0].stdout.strip()
    print(output)

### Envoyez une Requête

In [ ]:
# Récupère l'adresse IP du master
master_ip = master_node[0].address 

# Remplacez 31192 par le port que vous avez obtenu à l'étape 1
node_port = 30968 # ⚠️ CHANGEZ CE PORT

# Test avec curl depuis le master
with en.actions(roles=master_node) as p:
    p.shell(f"curl http://{master_ip}:{node_port}")

if p.results:
    print("--- Réponse du serveur ---")
    print(p.results[0].stdout)

# Installation de la Suite de Monitoring (Prometheus & Grafana)

In [ ]:
# Script pour déployer la suite de monitoring
MONITORING_SCRIPT = """
#!/bin/bash -e

echo "--- Installation de la suite Prometheus + Grafana via kube-prometheus ---"

# 1. Cloner le dépôt qui contient les manifestes
if [ ! -d "kube-prometheus" ]; then
    echo "Clonage du dépôt kube-prometheus..."
    git clone https://github.com/prometheus-operator/kube-prometheus.git
else
    echo "Dépôt kube-prometheus déjà présent."
fi
cd kube-prometheus

# 2. Appliquer les définitions de ressources (CRDs) et la configuration de base
echo "Étape 1/3 : Application des CRDs et de la configuration 'setup'..."
kubectl apply --server-side -f manifests/setup

# Attendre que les CRDs soient bien enregistrés dans le cluster avant de continuer
echo "Attente de l'établissement des CRDs..."
kubectl wait --for condition=Established --all CustomResourceDefinition --timeout=300s

# 3. Appliquer le reste de la suite (Prometheus, Grafana, Alertmanager, etc.)
echo "Étape 2/3 : Application des manifestes de la suite de monitoring..."
kubectl apply -f manifests/

# 4. Attendre que les déploiements clés soient prêts dans le namespace 'monitoring'
echo "Étape 3/3 : Attente du démarrage de Prometheus et Grafana (peut prendre quelques minutes)..."
kubectl wait --for=condition=available deployment/prometheus-k8s -n monitoring --timeout=300s
kubectl wait --for=condition=available deployment/grafana -n monitoring --timeout=300s

echo ""
echo "--- Vérification des pods dans le namespace 'monitoring' ---"
kubectl get pods -n monitoring

echo "\n🎉 Suite de monitoring Prometheus et Grafana installée avec succès !"
"""

# Exécution du script sur le master
print("Lancement de l'installation de la suite de monitoring...")
with en.actions(roles=master_node) as p:
    p.shell(MONITORING_SCRIPT)

# Affichage du résultat
print("\n--- Résultat de l'installation ---")
if p.results:
    output = p.results[0].stdout.strip()
    print(output)

## Vérification des Composants de Monitoring

In [ ]:
# Script pour lister les pods et services de monitoring
VERIFY_MONITORING_SCRIPT = """
#!/bin/bash

echo "--- Pods dans le namespace 'monitoring' ---"
kubectl get pods -n monitoring -o wide

echo ""
echo "--- Services dans le namespace 'monitoring' ---"
kubectl get services -n monitoring
echo ""
echo "--- ServiceAccounts dans le namespace 'monitoring' ---"
kubectl get serviceaccounts -n monitoring
"""

# Exécution du script de vérification sur le master
print("Récupération de l'état des composants de monitoring...")
with en.actions(roles=master_node) as p:
    p.shell(VERIFY_MONITORING_SCRIPT)

# Affichage du résultat
if p.results:
    output = p.results[0].stdout.strip()
    print(output)

## Mise à Jour des NetworkPolicies pour l'Accès Externe

In [ ]:
# Script pour mettre à jour les NetworkPolicies
UPDATE_NP_SCRIPT = """
#!/bin/bash -e

echo "--- 1. Mise à jour de la NetworkPolicy de Grafana ---"
cat <<EOF | kubectl apply -f -
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: grafana
  namespace: monitoring
  labels:
    app.kubernetes.io/component: grafana
    app.kubernetes.io/name: grafana
    app.kubernetes.io/part-of: kube-prometheus
    app.kubernetes.io/version: 12.2.0
spec:
  podSelector:
    matchLabels:
      app.kubernetes.io/component: grafana
      app.kubernetes.io/name: grafana
      app.kubernetes.io/part-of: kube-prometheus
  policyTypes:
  - Ingress
  - Egress
  ingress:
  - from:
    # On autorise le trafic depuis n'importe quelle adresse IP (externe/interne)
    - ipBlock:
        cidr: 0.0.0.0/0
    # On garde la règle existante qui autorise Prometheus
    - podSelector:
        matchLabels:
          app.kubernetes.io/name: prometheus
    ports:
    - port: 3000
      protocol: TCP
  egress:
  - {}
EOF

echo "\n--- 2. Mise à jour de la NetworkPolicy de Prometheus ---"
cat <<EOF | kubectl apply -f -
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: prometheus-k8s
  namespace: monitoring
  labels:
    app.kubernetes.io/component: prometheus
    app.kubernetes.io/instance: k8s
    app.kubernetes.io/name: prometheus
    app.kubernetes.io/part-of: kube-prometheus
    app.kubernetes.io/version: 3.6.0
spec:
  podSelector:
    matchLabels:
      app.kubernetes.io/component: prometheus
      app.kubernetes.io/instance: k8s
      app.kubernetes.io/name: prometheus
      app.kubernetes.io/part-of: kube-prometheus
  policyTypes:
  - Ingress
  - Egress
  ingress:
  - from:
    # On autorise le trafic depuis n'importe quelle adresse IP
    - ipBlock:
        cidr: 0.0.0.0/0
    # On garde les règles existantes pour la communication interne
    - namespaceSelector: {}
    - podSelector:
        matchLabels:
          app.kubernetes.io/name: prometheus
    - podSelector:
        matchLabels:
          app.kubernetes.io/name: prometheus-adapter
    - podSelector:
        matchLabels:
          app.kubernetes.io/name: grafana
    ports:
    - port: 9090
      protocol: TCP
    - port: 8080
      protocol: TCP
  egress:
  - {}
EOF

echo "\nNetworkPolicies mises à jour avec succès pour autoriser l'accès externe."
"""

# Exécution du script sur le master
print("Mise à jour des NetworkPolicies...")
with en.actions(roles=master_node) as p:
    p.shell(UPDATE_NP_SCRIPT)

# Affichage du résultat
if p.results:
    output = p.results[0].stdout.strip()
    print(output)

## Passage des Services en NodePort

In [ ]:
# Script pour patcher les services en NodePort
PATCH_SERVICES_SCRIPT = """
#!/bin/bash -e

echo "--- 1. Modification du service Grafana en type NodePort ---"
kubectl patch service grafana -n monitoring -p '{"spec": {"type": "NodePort"}}'

echo ""
echo "--- 2. Modification du service Prometheus en type NodePort ---"
kubectl patch service prometheus-k8s -n monitoring -p '{"spec": {"type": "NodePort"}}'

echo ""
echo "--- 3. Vérification du résultat ---"
# Petite pause pour s'assurer que l'API server a bien traité les changements
sleep 2
kubectl get services -n monitoring
"""

# Exécution du script sur le master
print("Patch des services Grafana et Prometheus en NodePort...")
with en.actions(roles=master_node) as p:
    p.shell(PATCH_SERVICES_SCRIPT)

# Affichage du résultat
print("\n--- Résultat de la commande patch ---")
if p.results:
    output = p.results[0].stdout.strip()
    print(output)

# Installer Kepler depuis les Manifestes Locaux

In [ ]:
import os

# --- Étape 1 : Copier le répertoire local 'kepler' vers le master ---

source_dir = "kepler"
dest_dir = "/tmp/kepler"

if not os.path.isdir(source_dir):
    raise FileNotFoundError(f"Le répertoire local '{source_dir}' est introuvable.")

print(f"--- Copie du répertoire local '{source_dir}' vers '{dest_dir}' sur le master... ---")
with en.actions(roles=master_node) as p:
    p.copy(src=source_dir, dest="/tmp/")

if p.results and p.results[0].status == "OK":
    print("Répertoire copié avec succès.")
else:
    print("Erreur lors de la copie du répertoire.")
    if p.results:
        print(p.results[0].stderr)


# --- Étape 2 : Appliquer les manifestes et vérifier ---

# Script simplifié : la création du namespace est gérée par les manifestes
APPLY_KEPLER_MANIFESTS_SCRIPT = f"""
#!/bin/bash -e

echo "--- Application de tous les manifestes depuis le répertoire {dest_dir} ---"
kubectl apply -f "{dest_dir}"

echo ""
echo "--- Attente de 30 secondes pour le démarrage des pods ---"
sleep 30

echo ""
echo "--- Vérification du statut des pods dans le namespace 'kepler' ---"
kubectl get pods -n kepler

echo "--- Vérification que le Service a bien été créé ---"
kubectl get svc -n kepler -o wide

echo ""
echo "🎉 Kepler a été déployé à partir des manifestes locaux."
"""

print(f"\n--- Application des manifestes sur le cluster... ---")
with en.actions(roles=master_node) as p:
    p.shell(APPLY_KEPLER_MANIFESTS_SCRIPT)

if p.results:
    output = p.results[0].stdout.strip()
    print(output)

## Diagnostic - Pourquoi Prometheus ne Scrape pas Kepler ?

In [ ]:
# Script de diagnostic complet pour identifier pourquoi Prometheus ne scrape pas Kepler
DIAGNOSTIC_KEPLER_PROMETHEUS = """
#!/bin/bash

echo "======================================================================"
echo "🔍 DIAGNOSTIC COMPLET: Prometheus ↔ Kepler"
echo "======================================================================"

echo ""
echo "--- 1. Vérification des Pods Kepler ---"
kubectl get pods -n kepler -o wide

echo ""
echo "--- 2. Vérification du Service Kepler ---"
kubectl get svc -n kepler -o wide

echo ""
echo "--- 3. Vérification des Labels du Service Kepler ---"
echo "Les labels sont CRITIQUES pour que le ServiceMonitor fonctionne!"
kubectl get svc -n kepler -o yaml | grep -A 10 "labels:"

echo ""
echo "--- 4. Vérification du ServiceMonitor Kepler ---"
kubectl get servicemonitor -n kepler -o wide

echo ""
echo "--- 5. Détails du ServiceMonitor (selector et labels) ---"
kubectl get servicemonitor -n kepler -o yaml | grep -A 20 "selector:"

echo ""
echo "--- 6. Vérification que Prometheus-Operator voit le ServiceMonitor ---"
echo "Prometheus doit être dans le même namespace ou avec les bonnes règles RBAC"
kubectl get servicemonitor -A

echo ""
echo "--- 7. Configuration de Prometheus pour ServiceMonitor ---"
echo "Vérification des serviceMonitorSelector dans Prometheus:"
kubectl get prometheus -n monitoring -o yaml | grep -A 5 "serviceMonitorSelector:"

echo ""
echo "--- 8. Vérification des Endpoints Kepler ---"
echo "Si pas d'endpoints, le service ne pointe vers rien!"
kubectl get endpoints -n kepler

echo ""
echo "--- 9. Test direct du endpoint Kepler ---"
KEPLER_POD=`kubectl get pods -n kepler -l app.kubernetes.io/name=kepler -o jsonpath='{.items[0].metadata.name}'`
if [ -n "$KEPLER_POD" ]; then
    echo "Test de l'endpoint metrics sur le pod $KEPLER_POD:"
    kubectl exec -n kepler $KEPLER_POD -- curl -s localhost:9102/metrics | head -20
else
    echo "❌ Aucun pod Kepler trouvé!"
fi

echo ""
echo "--- 10. Vérification des Logs Prometheus ---"
echo "Recherche d'erreurs liées à Kepler dans les logs Prometheus:"
PROM_POD=`kubectl get pods -n monitoring -l app.kubernetes.io/name=prometheus -o jsonpath='{.items[0].metadata.name}'`
if [ -n "$PROM_POD" ]; then
    kubectl logs -n monitoring $PROM_POD -c prometheus --tail=50 | grep -i kepler || echo "Aucune mention de Kepler dans les logs récents"
else
    echo "❌ Pod Prometheus non trouvé!"
fi

echo ""
echo "--- 11. Configuration Actuelle des Targets Prometheus ---"
echo "Vérifiez si Kepler apparaît dans les targets de Prometheus"
echo "💡 Accédez à l'UI Prometheus: Status → Targets"
echo "💡 Ou utilisez l'API: curl http://prometheus-ip:port/api/v1/targets"

echo ""
echo "======================================================================"
echo "🎯 CAUSES COMMUNES ET SOLUTIONS:"
echo "======================================================================"
echo ""
echo "❌ PROBLÈME 1: Labels incompatibles"
echo "   Le ServiceMonitor cherche des labels spécifiques sur le Service"
echo "   Solution: Vérifiez que les labels du Service matchent le selector du ServiceMonitor"
echo ""
echo "❌ PROBLÈME 2: Namespace incorrect"
echo "   Le ServiceMonitor doit être dans un namespace que Prometheus surveille"
echo "   Solution: Soit mettre le ServiceMonitor dans 'monitoring', soit configurer Prometheus"
echo ""
echo "❌ PROBLÈME 3: serviceMonitorSelector vide"
echo "   Prometheus n'est configuré pour surveiller AUCUN ServiceMonitor"
echo "   Solution: Configurer prometheus.serviceMonitorSelector: {}"
echo ""
echo "❌ PROBLÈME 4: Port incorrect"
echo "   Le ServiceMonitor pointe vers un port qui n'existe pas"
echo "   Solution: Vérifier que le port dans ServiceMonitor = port du Service"
echo ""
echo "======================================================================"
"""

# --- Enregistrer le script dans un fichier local ---
script_path = "diagnostic_kepler.sh"
with open(script_path, "w") as f:
    f.write(DIAGNOSTIC_KEPLER_PROMETHEUS)
os.chmod(script_path, 0o755)

print("🔍 Lancement du diagnostic complet Prometheus ↔ Kepler...")
with en.actions(roles=master_node) as p:
    p.script(script_path)

if p.results:
    output = p.results[0].stdout.strip()
    print(output)

## 🔧 Solution : Donner les Permissions RBAC à Prometheus pour Kepler

In [ ]:
# Script pour corriger les permissions RBAC de Prometheus
FIX_PROMETHEUS_RBAC = """
#!/bin/bash -e

echo "========================================================================"
echo "🔧 Correction des Permissions RBAC pour Prometheus → Kepler"
echo "========================================================================"

echo ""
echo "--- Création d'un ClusterRole avec les permissions nécessaires ---"
cat <<EOF | kubectl apply -f -
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  name: prometheus-kepler-access
rules:
- apiGroups: [""]
  resources:
  - services
  - endpoints
  - pods
  verbs: ["get", "list", "watch"]
- apiGroups: ["discovery.k8s.io"]
  resources:
  - endpointslices
  verbs: ["get", "list", "watch"]
EOF

echo ""
echo "--- Liaison du ClusterRole au ServiceAccount prometheus-k8s ---"
cat <<EOF | kubectl apply -f -
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: prometheus-kepler-access
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: prometheus-kepler-access
subjects:
- kind: ServiceAccount
  name: prometheus-k8s
  namespace: monitoring
EOF

echo ""
echo "--- Vérification des permissions créées ---"
kubectl get clusterrole prometheus-kepler-access
kubectl get clusterrolebinding prometheus-kepler-access

echo ""
echo "--- Redémarrage des pods Prometheus pour appliquer les changements ---"
kubectl rollout restart statefulset prometheus-k8s -n monitoring

echo ""
echo "--- Attente du redémarrage de Prometheus (30 secondes) ---"
sleep 30

echo ""
echo "--- Vérification que Prometheus est prêt ---"
kubectl wait --for=condition=ready pod -l app.kubernetes.io/name=prometheus -n monitoring --timeout=120s

echo ""
echo "========================================================================"
echo "✅ Permissions RBAC configurées avec succès !"
echo "========================================================================"
echo ""
echo "Prometheus peut maintenant accéder aux EndpointSlices du namespace kepler."
echo "Attendez 1-2 minutes puis vérifiez les targets dans l'UI Prometheus."
"""

print("🔧 Application de la correction RBAC...")
with en.actions(roles=master_node) as p:
    p.shell(FIX_PROMETHEUS_RBAC)

if p.results:
    output = p.results[0].stdout.strip()
    print(output)

# Déploiement des Ressources PowerCap

In [ ]:
import os

# --- Script pour déployer tous les manifestes PowerCap ---

source_dir = "powercap"
dest_dir = "/tmp/powercap"

# Vérifier que le répertoire local existe
if not os.path.isdir(source_dir):
    print(f"⚠️  Le répertoire local '{source_dir}' n'existe pas.")
    print(f"📁 Veuillez créer le dossier '{source_dir}' avec les fichiers suivants:")
    print("   - daemonset.yaml")
    print("   - rbac.yaml")
    print("   - configmap.yaml")
    print("\n💡 Ou uploadez le dossier complet dans le répertoire de travail.")
    raise FileNotFoundError(f"Le répertoire local '{source_dir}' est introuvable. Créez-le d'abord avec les manifestes nécessaires.")

# --- Étape 1 : Copier le répertoire local vers le master ---
print(f"--- Copie du répertoire '{source_dir}' vers '{dest_dir}' sur le master... ---")
with en.actions(roles=master_node) as p:
    p.copy(src=source_dir, dest="/tmp/")

if p.results and p.results[0].status == "OK":
    print("✅ Répertoire copié avec succès.")
else:
    print("❌ Erreur lors de la copie du répertoire.")
    if p.results:
        print(p.results[0].stderr)

# --- Étape 2 : Appliquer tous les manifestes ---
DEPLOY_POWERCAP_SCRIPT = f"""
#!/bin/bash -e

echo "========================================================================"
echo "🚀 Déploiement des Ressources PowerCap dans le namespace 'default'"
echo "========================================================================"

echo ""
echo "--- Application de tous les manifestes depuis {dest_dir} ---"
kubectl apply -f "{dest_dir}"

# echo ""
# echo "--- Attente de 30 secondes pour la création des ressources ---"
# sleep 30

echo ""
echo "--- Vérification du DaemonSet powercap-manager ---"
kubectl get daemonset powercap-manager -n default -o wide

echo ""
echo "--- Vérification des pods powercap-manager ---"
kubectl get pods -n default -l app=powercap-manager -o wide

echo ""
echo "--- Statut du DaemonSet (nombre de pods prêts) ---"
kubectl rollout status daemonset/powercap-manager -n default --timeout=60s

echo ""
echo "--- Vérification du ServiceAccount ---"
kubectl get serviceaccount powercap-manager -n default

echo ""
echo "--- Vérification de la ConfigMap ---"
kubectl get configmap powercap-config -n default

echo ""
echo "--- Vérification des services (si présents) ---"
kubectl get services -n default -l app=powercap-manager || echo "Aucun service trouvé"

echo ""
echo "========================================================================"
echo "✅ Déploiement PowerCap terminé !"
echo "========================================================================"
"""

print("\n--- Déploiement des ressources PowerCap... ---")
with en.actions(roles=master_node) as p:
    p.shell(DEPLOY_POWERCAP_SCRIPT)

if p.results:
    output = p.results[0].stdout.strip()
    print(output)

## Vérification de l'État et des Logs PowerCap

In [ ]:
# Script pour vérifier l'état détaillé et afficher les logs
CHECK_POWERCAP_STATUS_LOGS = """
#!/bin/bash

echo "========================================================================"
echo "📊 VÉRIFICATION DÉTAILLÉE - État et Logs PowerCap Manager"
echo "========================================================================"

echo ""
echo "--- 1. État du DaemonSet powercap-manager ---"
kubectl get daemonset powercap-manager -n default -o wide

echo ""
echo "--- 2. Détails du DaemonSet ---"
kubectl describe daemonset powercap-manager -n default | head -50

echo ""
echo "--- 3. Liste des pods powercap-manager sur tous les nœuds ---"
kubectl get pods -n default -l app=powercap-manager -o wide

echo ""
echo "--- 4. Nombre de pods actifs par rapport au nombre attendu ---"
DESIRED=$(kubectl get daemonset powercap-manager -n default -o jsonpath='{.status.desiredNumberScheduled}')
READY=$(kubectl get daemonset powercap-manager -n default -o jsonpath='{.status.numberReady}')
AVAILABLE=$(kubectl get daemonset powercap-manager -n default -o jsonpath='{.status.numberAvailable}')
echo "Pods désirés: $DESIRED"
echo "Pods prêts: $READY"
echo "Pods disponibles: $AVAILABLE"

echo ""
echo "--- 5. Description détaillée de chaque pod ---"
POWERCAP_PODS=$(kubectl get pods -n default -l app=powercap-manager -o jsonpath='{.items[*].metadata.name}')

if [ -n "$POWERCAP_PODS" ]; then
    for pod in $POWERCAP_PODS; do
        echo ""
        echo "=========================================="
        echo "Pod: $pod"
        echo "=========================================="
        
        echo ""
        echo "--- État général ---"
        kubectl get pod $pod -n default -o wide
        
        echo ""
        echo "--- Nœud d'exécution ---"
        NODE=$(kubectl get pod $pod -n default -o jsonpath='{.spec.nodeName}')
        echo "Nœud: $NODE"
        
        echo ""
        echo "--- Statut du conteneur ---"
        kubectl get pod $pod -n default -o jsonpath='{range .status.containerStatuses[*]}Conteneur: {.name}
Ready: {.ready}
Restarts: {.restartCount}
Image: {.image}
{end}'
        
        echo ""
        echo "--- Conditions du pod ---"
        kubectl get pod $pod -n default -o jsonpath='{range .status.conditions[*]}{.type}: {.status} ({.reason})
{end}'
    done
else
    echo "❌ Aucun pod powercap-manager trouvé dans le namespace default"
fi

echo ""
echo "--- 6. Logs de tous les pods powercap-manager ---"
if [ -n "$POWERCAP_PODS" ]; then
    for pod in $POWERCAP_PODS; do
        echo ""
        echo "=========================================="
        echo "📝 LOGS du pod: $pod"
        echo "=========================================="
        kubectl logs $pod -n default --tail=100 2>&1 || echo "Impossible de récupérer les logs"
    done
else
    echo "❌ Aucun log à afficher"
fi

echo ""
echo "--- 7. Événements récents liés à powercap-manager ---"
kubectl get events -n default --sort-by='.lastTimestamp' | grep powercap | tail -30 || echo "Aucun événement récent"

echo ""
echo "--- 8. Vérification de l'accès aux ressources système ---"
if [ -n "$POWERCAP_PODS" ]; then
    # Test sur le premier pod
    FIRST_POD=$(echo $POWERCAP_PODS | awk '{print $1}')
    echo "Test d'accès RAPL sur le pod: $FIRST_POD"
    kubectl exec $FIRST_POD -n default -- ls -la /sys/devices/virtual/powercap/ 2>&1 || echo "❌ Impossible d'accéder au répertoire RAPL"
fi

echo ""
echo "--- 9. Vérification de la ConfigMap ---"
kubectl get configmap powercap-config -n default -o yaml

echo ""
echo "--- 10. Vérification des erreurs critiques ---"
if [ -n "$POWERCAP_PODS" ]; then
    for pod in $POWERCAP_PODS; do
        STATUS=$(kubectl get pod $pod -n default -o jsonpath='{.status.phase}')
        if [ "$STATUS" != "Running" ]; then
            echo ""
            echo "⚠️  ATTENTION: Pod $pod est en état: $STATUS"
            echo "Raison:"
            kubectl get pod $pod -n default -o jsonpath='{.status.conditions[?(@.type=="Ready")].message}'
            echo ""
            echo "Logs d'erreur:"
            kubectl logs $pod -n default --tail=20 2>&1
        fi
    done
fi

echo ""
echo "========================================================================"
echo "✅ Vérification terminée"
echo "========================================================================"
"""

print("🔍 Lancement de la vérification détaillée de l'état et des logs PowerCap...")
with en.actions(roles=master_node) as p:
    p.shell(CHECK_POWERCAP_STATUS_LOGS)

if p.results:
    output = p.results[0].stdout.strip()
    print(output)

## Vérification des Annotations des Nœuds par PowerCap

In [ ]:
# Script pour vérifier les annotations ajoutées par PowerCap sur les nœuds
CHECK_NODE_ANNOTATIONS_SCRIPT = """
#!/bin/bash

echo "========================================================================"
echo "🏷️  VÉRIFICATION DES ANNOTATIONS DES NŒUDS"
echo "========================================================================"

echo ""
echo "--- Liste de tous les nœuds du cluster ---"
kubectl get nodes

echo ""
echo "--- Annotations complètes de tous les nœuds ---"
for node in $(kubectl get nodes -o jsonpath='{.items[*].metadata.name}'); do
    echo ""
    echo "=========================================="
    echo "📍 Nœud: $node"
    echo "=========================================="
    
    echo ""
    echo "--- Type de nœud ---"
    kubectl get node $node -o jsonpath='{.metadata.labels.node-role\.kubernetes\.io/control-plane}' > /dev/null 2>&1 && echo "Type: Master/Control-Plane" || echo "Type: Worker"
    
    echo ""
    echo "--- Toutes les annotations ---"
    kubectl get node $node -o jsonpath='{.metadata.annotations}' | jq '.' 2>/dev/null || kubectl get node $node -o jsonpath='{.metadata.annotations}'
    
    echo ""
    echo "--- Annotations spécifiques à PowerCap (filtrées) ---"
    kubectl get node $node -o json | jq -r '.metadata.annotations | to_entries[] | select(.key | contains("power") or contains("rapl") or contains("energy") or contains("cap")) | "\(.key): \(.value)"' 2>/dev/null || echo "Aucune annotation PowerCap trouvée ou jq non disponible"
done

echo ""
echo "--- Récapitulatif : Annotations PowerCap par nœud ---"
kubectl get nodes -o custom-columns=NAME:.metadata.name,ANNOTATIONS:.metadata.annotations 2>/dev/null || echo "Format custom-columns non supporté"

echo ""
echo "========================================================================"
echo "✅ Vérification des annotations terminée"
echo "========================================================================"
"""

print("🏷️  Vérification des annotations des nœuds...")
with en.actions(roles=master_node) as p:
    p.shell(CHECK_NODE_ANNOTATIONS_SCRIPT)

if p.results:
    output = p.results[0].stdout.strip()
    print(output)

## Suppression du DaemonSet PowerCap

In [ ]:
import os

# Vérifier si le dossier local existe, sinon afficher une erreur
source_dir = "powercap"
dest_dir = "/tmp/powercap"

if not os.path.isdir(source_dir):
    print(f"❌ Le répertoire local '{source_dir}' n'existe pas.")
    print("💡 Créez le dossier avec les manifestes avant de supprimer les ressources.")
    raise FileNotFoundError(f"Le répertoire '{source_dir}' est introuvable.")

# --- Copier le répertoire vers le master (au cas où il n'existe plus sur le master) ---
print(f"📁 Vérification/copie du répertoire '{source_dir}' vers le master...")
with en.actions(roles=master_node) as p:
    p.copy(src=source_dir, dest="/tmp/")

if p.results and p.results[0].status == "OK":
    print("✅ Répertoire copié/mis à jour sur le master.")

# Script pour supprimer complètement le DaemonSet PowerCap en utilisant les manifestes
DELETE_POWERCAP_SCRIPT = f"""
#!/bin/bash

echo "========================================================================"
echo "🗑️  SUPPRESSION des Ressources PowerCap"
echo "========================================================================"

echo ""
echo "--- Suppression de toutes les ressources via les manifestes ---"
kubectl delete -f {dest_dir}/ --ignore-not-found=true

echo ""
echo "--- Attente de la suppression des pods (10 secondes) ---"
sleep 10

echo ""
echo "--- Suppression des annotations PowerCap sur les nœuds ---"
for node in $(kubectl get nodes -o jsonpath='{{.items[*].metadata.name}}'); do
    echo "Nettoyage des annotations sur le nœud: $node"
    
    # Suppression de chaque annotation PowerCap (avec 2>&1 pour voir les erreurs)
    kubectl annotate node $node power-manager/initialized- 2>&1 || true
    kubectl annotate node $node rapl/last-update- 2>&1 || true
    kubectl annotate node $node rapl/market-period- 2>&1 || true
    kubectl annotate node $node rapl/market-price- 2>&1 || true
    kubectl annotate node $node rapl/market-volume- 2>&1 || true
    kubectl annotate node $node rapl/max_power_uw- 2>&1 || true
    kubectl annotate node $node rapl/pmax- 2>&1 || true
    kubectl annotate node $node rapl/provider- 2>&1 || true
done
echo "✅ Annotations PowerCap supprimées de tous les nœuds"

echo ""
echo "--- Vérification finale ---"
echo ""
echo "DaemonSets restants avec 'powercap' :"
kubectl get daemonsets -n default | grep powercap || echo "✅ Aucun DaemonSet trouvé"

echo ""
echo "Pods restants avec 'powercap' :"
kubectl get pods -n default -l app=powercap-manager || echo "✅ Aucun pod trouvé"

echo ""
echo "ConfigMaps restantes avec 'powercap' :"
kubectl get configmap -n default | grep powercap || echo "✅ Aucune ConfigMap trouvée"

echo ""
echo "ServiceAccounts restants avec 'powercap' :"
kubectl get serviceaccount -n default | grep powercap || echo "✅ Aucun ServiceAccount trouvé"

echo ""
echo "ClusterRoles restants avec 'powercap' :"
kubectl get clusterrole | grep powercap || echo "✅ Aucun ClusterRole trouvé"

echo ""
echo "ClusterRoleBindings restants avec 'powercap' :"
kubectl get clusterrolebinding | grep powercap || echo "✅ Aucun ClusterRoleBinding trouvé"

echo ""
echo "--- Vérification des annotations restantes ---"
echo "Annotations PowerCap/RAPL restantes sur les nœuds:"
for node in $(kubectl get nodes -o jsonpath='{{.items[*].metadata.name}}'); do
    echo ""
    echo "Nœud: $node"
    kubectl get node $node -o json | jq -r '.metadata.annotations | to_entries[] | select(.key | contains("power") or contains("rapl")) | "  \\(.key): \\(.value)"' 2>/dev/null || echo "  Aucune annotation PowerCap/RAPL trouvée"
done

echo ""
echo "========================================================================"
echo "✅ Suppression de PowerCap terminée !"
echo "========================================================================"
"""

print("🗑️  Lancement de la suppression de PowerCap...")
with en.actions(roles=master_node) as p:
    p.shell(DELETE_POWERCAP_SCRIPT)

if p.results:
    output = p.results[0].stdout.strip()
    print(output)

# --- Nettoyage des données sur les workers ---
print("\n--- Nettoyage des données sur les nœuds workers ---")
CLEANUP_WORKERS_SCRIPT = """
#!/bin/bash
echo "Suppression des données PowerCap dans /var/lib/powercap..."
sudo rm -rf /var/lib/powercap/*.csv 2>/dev/null || true
echo "✅ Nettoyage terminé sur ce nœud"
"""

with en.actions(roles=worker_nodes) as p:
    p.shell(CLEANUP_WORKERS_SCRIPT)

print("✅ Toutes les ressources PowerCap ont été supprimées du cluster !")

# Déploiement de Volta

In [ ]:
import os

# --- Script pour déployer Volta ---

source_dir = "volta"
config_file = "volta-config.yaml"
manifest_file = "volta.yaml"

# Vérifier que le répertoire local existe
if not os.path.isdir(source_dir):
    print(f"❌ Le répertoire local '{source_dir}' n'existe pas.")
    print(f"📁 Veuillez créer le dossier '{source_dir}' avec les fichiers suivants:")
    print(f"   - {config_file}")
    print(f"   - {manifest_file}")
    print("\n💡 Ou uploadez le dossier complet dans le répertoire de travail.")
    raise FileNotFoundError(f"Le répertoire local '{source_dir}' est introuvable. Créez-le d'abord avec les manifestes nécessaires.")

# Vérifier que les fichiers nécessaires existent
config_path = os.path.join(source_dir, config_file)
manifest_path = os.path.join(source_dir, manifest_file)

if not os.path.isfile(config_path):
    raise FileNotFoundError(f"Le fichier de configuration '{config_path}' est introuvable.")

if not os.path.isfile(manifest_path):
    raise FileNotFoundError(f"Le fichier manifest '{manifest_path}' est introuvable.")

print(f"✅ Fichiers Volta trouvés:")
print(f"   - Configuration: {config_path}")
print(f"   - Manifest: {manifest_path}")

# --- Étape 1 : Copier le fichier de configuration vers /etc/kubernetes/ sur le master ---
print(f"\n--- Copie du fichier de configuration vers /etc/kubernetes/ sur le master... ---")
with en.actions(roles=master_node) as p:
    p.copy(src=config_path, dest="/etc/kubernetes/volta-config.yaml")

if p.results and p.results[0].status == "OK":
    print("✅ Fichier de configuration copié avec succès.")
else:
    print("❌ Erreur lors de la copie du fichier de configuration.")
    if p.results:
        print(p.results[0].stderr)

# --- Étape 2 : Copier le manifest vers le master ---
print(f"\n--- Copie du manifest Volta vers le master... ---")
with en.actions(roles=master_node) as p:
    p.copy(src=manifest_path, dest="/tmp/volta.yaml")

if p.results and p.results[0].status == "OK":
    print("✅ Manifest Volta copié avec succès.")
else:
    print("❌ Erreur lors de la copie du manifest.")
    if p.results:
        print(p.results[0].stderr)

# --- Étape 3 : Déployer Volta ---
DEPLOY_VOLTA_SCRIPT = """
#!/bin/bash -e

echo "========================================================================"
echo "⚡ Déploiement de Volta Scheduler"
echo "========================================================================"

echo ""
echo "--- Vérification du fichier de configuration ---"
if [ -f "/etc/kubernetes/volta-config.yaml" ]; then
    echo "✅ Fichier de configuration trouvé: /etc/kubernetes/volta-config.yaml"
    ls -la /etc/kubernetes/volta-config.yaml
    echo ""
    echo "Contenu du fichier de configuration:"
    head -20 /etc/kubernetes/volta-config.yaml
else
    echo "❌ Fichier de configuration non trouvé!"
    exit 1
fi

echo ""
echo "--- Application du manifest Volta ---"
kubectl apply -f /tmp/volta.yaml

echo ""
echo "--- Attente de 30 secondes pour le démarrage ---"
sleep 30

echo ""
echo "--- Vérification du déploiement volta-scheduler ---"
kubectl get deployment volta-scheduler -n kube-system -o wide

echo ""
echo "--- Vérification des pods Volta (component=scheduler) ---"
kubectl get pods -n kube-system -l component=scheduler,tier=control-plane -o wide

echo ""
echo "--- Statut du déploiement Volta ---"
kubectl rollout status deployment/volta-scheduler -n kube-system --timeout=120s

echo ""
echo "--- Vérification de toutes les ressources liées à Volta ---"
kubectl get all -n kube-system | grep volta || echo "Aucune ressource Volta trouvée"

echo ""
echo "--- Événements récents liés à volta-scheduler ---"
kubectl get events -n kube-system --sort-by='.lastTimestamp' | grep volta | tail -20 || echo "Aucun événement Volta récent"

echo ""
echo "--- Logs des pods volta-scheduler ---"
kubectl logs -l component=scheduler,tier=control-plane -n kube-system --tail=30 2>&1 || echo "Aucun log disponible"

echo ""
echo "========================================================================"
echo "✅ Déploiement de Volta Scheduler terminé !"
echo "========================================================================"
"""

print("\n--- Déploiement de Volta... ---")
with en.actions(roles=master_node) as p:
    p.shell(DEPLOY_VOLTA_SCRIPT)

if p.results:
    output = p.results[0].stdout.strip()
    print(output)

## Récupération de Tous les Logs Volta

In [ ]:
# Script pour récupérer tous les logs de Volta Scheduler
GET_ALL_VOLTA_LOGS_SCRIPT = """
#!/bin/bash

echo "========================================================================"
echo "📋 RÉCUPÉRATION COMPLÈTE DES LOGS VOLTA SCHEDULER"
echo "========================================================================"

echo ""
echo "--- 1. Recherche de tous les pods volta-scheduler ---"
VOLTA_PODS=$(kubectl get pods -n kube-system -l component=scheduler,tier=control-plane -o jsonpath='{.items[*].metadata.name}' | grep volta)

if [ -z "$VOLTA_PODS" ]; then
    echo "❌ Aucun pod volta-scheduler trouvé"
    exit 1
fi

echo "Pods Volta trouvés: $VOLTA_PODS"

echo ""
echo "--- 2. Informations détaillées sur les pods Volta ---"
for pod in $VOLTA_PODS; do
    echo ""
    echo "=========================================="
    echo "Pod: $pod"
    echo "=========================================="
    
    echo "--- État du pod ---"
    kubectl get pod $pod -n kube-system -o wide
    
    echo ""
    echo "--- Nœud d'exécution ---"
    kubectl get pod $pod -n kube-system -o jsonpath='{.spec.nodeName}'
    echo ""
    
    echo ""
    echo "--- Statut des conteneurs ---"
    kubectl get pod $pod -n kube-system -o jsonpath='{range .status.containerStatuses[*]}Conteneur: {.name}, Ready: {.ready}, Restarts: {.restartCount}{"\n"}{end}'
done

echo ""
echo "--- 3. LOGS COMPLETS DE TOUS LES PODS VOLTA ---"
for pod in $VOLTA_PODS; do
    echo ""
    echo "=========================================="
    echo "📝 LOGS COMPLETS du pod: $pod"
    echo "=========================================="
    
    # Logs depuis le début (pas de --tail)
    kubectl logs $pod -n kube-system --timestamps 2>&1 || echo "❌ Impossible de récupérer les logs"
    
    echo ""
    echo "--- Logs du conteneur précédent (si redémarrage) ---"
    kubectl logs $pod -n kube-system --previous --timestamps 2>/dev/null || echo "Aucun log de conteneur précédent"
done

echo ""
echo "--- 4. LOGS EN TEMPS RÉEL (suivre les logs) ---"
echo "💡 Pour suivre les logs en temps réel, utilisez:"
echo "kubectl logs -f -l component=scheduler,tier=control-plane -n kube-system | grep volta"

echo ""
echo "--- 5. ÉVÉNEMENTS LIÉS À VOLTA ---"
kubectl get events -n kube-system --sort-by='.lastTimestamp' | grep -i volta | tail -50 || echo "Aucun événement Volta trouvé"

echo ""
echo "--- 6. DESCRIPTION DÉTAILLÉE DES PODS VOLTA ---"
for pod in $VOLTA_PODS; do
    echo ""
    echo "=========================================="
    echo "📋 DESCRIPTION du pod: $pod"
    echo "=========================================="
    kubectl describe pod $pod -n kube-system
done

echo ""
echo "--- 7. CONFIGURATION DU DÉPLOIEMENT VOLTA ---"
kubectl get deployment volta-scheduler -n kube-system -o yaml 2>&1 || echo "Déploiement Volta non trouvé"

echo ""
echo "--- 8. LOGS DE DÉBOGAGE - DÉCISIONS DE SCHEDULING ---"
echo "Recherche des logs de scheduling dans tous les pods volta:"
for pod in $VOLTA_PODS; do
    echo ""
    echo "--- Décisions de scheduling dans $pod ---"
    kubectl logs $pod -n kube-system | grep -i "schedule\|bind\|filter\|score" | tail -20 || echo "Aucun log de scheduling trouvé"
done

echo ""
echo "========================================================================"
echo "✅ Récupération complète des logs Volta terminée"
echo "========================================================================"
"""

print("📋 Récupération de tous les logs Volta...")
with en.actions(roles=master_node) as p:
    p.shell(GET_ALL_VOLTA_LOGS_SCRIPT)

if p.results:
    output = p.results[0].stdout.strip()
    print(output)

## Déploiement d'Application Demo avec Volta Scheduler

In [ ]:
import os

# --- Script pour déployer l'application demo avec Volta Scheduler ---

demo_dir = "demo"
demo_file = "rancher-demo.yaml"

# Vérifier que le répertoire et le fichier existent
if not os.path.isdir(demo_dir):
    print(f"❌ Le répertoire local '{demo_dir}' n'existe pas.")
    raise FileNotFoundError(f"Le répertoire '{demo_dir}' est introuvable.")

demo_path = os.path.join(demo_dir, demo_file)
if not os.path.isfile(demo_path):
    print(f"❌ Le fichier '{demo_path}' n'existe pas.")
    raise FileNotFoundError(f"Le fichier '{demo_path}' est introuvable.")

print(f"✅ Fichier demo trouvé: {demo_path}")


# --- Copier le fichier demo vers le master ---
print(f"\n--- Copie du fichier demo vers le master... ---")
with en.actions(roles=master_node) as p:
    p.copy(src=demo_path, dest="/tmp/rancher-demo.yaml")

if p.results and p.results[0].status == "OK":
    print("✅ Fichier demo copié avec succès.")
else:
    print("❌ Erreur lors de la copie du fichier demo.")
    if p.results:
        print(p.results[0].stderr)

# --- Appliquer le manifest demo avec Volta Scheduler ---
DEPLOY_DEMO_WITH_VOLTA_SCRIPT = """
#!/bin/bash -e

echo "========================================================================"
echo "🚀 Déploiement de l'Application Demo avec Volta Scheduler"
echo "========================================================================"

echo ""
echo "--- Application du manifest rancher-demo.yaml ---"
kubectl apply -f /tmp/rancher-demo.yaml

echo ""
echo "--- Attente de 30 secondes pour le démarrage ---"
sleep 30

echo ""
echo "--- Vérification du déploiement hello-world ---"
kubectl get deployment hello-world -o wide

echo ""
echo "--- Vérification des pods hello-world ---"
kubectl get pods -l app=hello-world -o wide

echo ""
echo "--- Vérification du scheduler utilisé ---"
kubectl get pods -l app=hello-world -o jsonpath='{range .items[*]}{.metadata.name}{"\t"}{.spec.schedulerName}{"\t"}{.spec.nodeName}{"\n"}{end}' | column -t

echo ""
echo "--- Statut du déploiement ---"
kubectl rollout status deployment/hello-world --timeout=120s

echo ""
echo "--- Événements récents liés à hello-world ---"
kubectl get events --sort-by='.lastTimestamp' | grep hello-world | tail -10 || echo "Aucun événement récent"

echo ""
echo "--- Logs du scheduler Volta (pour voir les décisions de placement) ---"
kubectl logs -l component=scheduler,tier=control-plane -n kube-system --tail=20 | grep -i "hello-world\|volta\|schedule" || echo "Aucun log de scheduling trouvé"

echo ""
echo "========================================================================"
echo "✅ Déploiement de l'application demo avec Volta terminé !"
echo "========================================================================"
"""

print("\n--- Déploiement de l'application demo avec Volta Scheduler... ---")
with en.actions(roles=master_node) as p:
    p.shell(DEPLOY_DEMO_WITH_VOLTA_SCRIPT)

if p.results:
    output = p.results[0].stdout.strip()
    print(output)

In [ ]:
DEPLOY_DEMO_WITH_VOLTA_SCRIPT = """
#!/bin/bash -e
echo "========================================================================"
echo ""
echo "--- Vérification du déploiement hello-world ---"
kubectl get deployment hello-world -o wide

echo ""
echo "--- Vérification des pods hello-world ---"
kubectl get pods -l app=hello-world -o wide

echo ""
echo "--- Vérification du scheduler utilisé ---"
kubectl get pods -l app=hello-world -o jsonpath='{range .items[*]}{.metadata.name}{"\t"}{.spec.schedulerName}{"\t"}{.spec.nodeName}{"\n"}{end}' | column -t

echo ""
echo "--- Statut du déploiement ---"
kubectl rollout status deployment/hello-world --timeout=120s

echo ""
echo "--- Événements récents liés à hello-world ---"
kubectl get events --sort-by='.lastTimestamp' | grep hello-world | tail -10 || echo "Aucun événement récent"

echo ""
echo "--- Logs du scheduler Volta (pour voir les décisions de placement) ---"
kubectl logs -l component=scheduler,tier=control-plane -n kube-system --tail=20 | grep -i "hello-world\|volta\|schedule" || echo "Aucun log de scheduling trouvé"

echo ""
echo "========================================================================"
echo "✅ Déploiement de l'application demo avec Volta terminé !"
echo "========================================================================"
"""

print("\n--- Déploiement de l'application demo avec Volta Scheduler... ---")
with en.actions(roles=master_node) as p:
    p.shell(DEPLOY_DEMO_WITH_VOLTA_SCRIPT)

if p.results:
    output = p.results[0].stdout.strip()
    print(output)

## Suppression de l'Application Demo

In [ ]:
# Script pour supprimer l'application demo
DELETE_DEMO_SCRIPT = """
#!/bin/bash

echo "========================================================================"
echo "🗑️  SUPPRESSION de l'Application Demo"
echo "========================================================================"

echo ""
echo "--- Suppression du déploiement hello-world ---"
kubectl delete deployment hello-world --ignore-not-found=true

echo ""
echo "--- Attente de la suppression des pods (10 secondes) ---"
sleep 10

echo ""
echo "--- Suppression des fichiers temporaires ---"
rm -f /tmp/rancher-demo.yaml /tmp/rancher-demo-volta.yaml 2>&1 || echo "Fichiers temporaires déjà supprimés"

echo ""
echo "--- Vérification finale ---"
echo ""
echo "Déploiements restants avec 'hello-world' :"
kubectl get deployments | grep hello-world || echo "✅ Aucun déploiement trouvé"

echo ""
echo "Pods restants avec 'hello-world' :"
kubectl get pods -l app=hello-world || echo "✅ Aucun pod trouvé"

echo ""
echo "========================================================================"
echo "✅ Suppression de l'application demo terminée !"
echo "========================================================================"
"""

print("🗑️  Lancement de la suppression de l'application demo...")
with en.actions(roles=master_node) as p:
    p.shell(DELETE_DEMO_SCRIPT)

if p.results:
    output = p.results[0].stdout.strip()
    print(output)

print("✅ L'application demo a été complètement supprimée du cluster !")

## Suppression de Volta Scheduler

In [ ]:
# Script pour supprimer complètement Volta Scheduler
DELETE_VOLTA_SCRIPT = """
#!/bin/bash

echo "========================================================================"
echo "🗑️  SUPPRESSION de Volta Scheduler"
echo "========================================================================"

echo ""
echo "--- Suppression du déploiement volta-scheduler ---"
kubectl delete deployment volta-scheduler -n kube-system --ignore-not-found=true

echo ""
echo "--- Attente de la suppression des pods (10 secondes) ---"
sleep 10

echo ""
echo "--- Suppression du fichier de configuration ---"
sudo rm -f /etc/kubernetes/volta-config.yaml 2>&1 || echo "Fichier de configuration déjà supprimé"

echo ""
echo "--- Suppression du manifest temporaire ---"
rm -f /tmp/volta.yaml 2>&1 || echo "Manifest temporaire déjà supprimé"

echo ""
echo "--- Vérification finale ---"
echo ""
echo "Déploiements restants avec 'volta' :"
kubectl get deployments -n kube-system | grep volta || echo "✅ Aucun déploiement trouvé"

echo ""
echo "Pods restants avec 'volta' :"
kubectl get pods -n kube-system | grep volta || echo "✅ Aucun pod trouvé"

echo ""
echo "Services restants avec 'volta' :"
kubectl get services -n kube-system | grep volta || echo "✅ Aucun service trouvé"

echo ""
echo "--- Vérification des fichiers de configuration ---"
ls -la /etc/kubernetes/volta-config.yaml 2>/dev/null || echo "✅ Fichier de configuration supprimé"

echo ""
echo "========================================================================"
echo "✅ Suppression de Volta Scheduler terminée !"
echo "========================================================================"
"""

print("🗑️  Lancement de la suppression de Volta Scheduler...")
with en.actions(roles=master_node) as p:
    p.shell(DELETE_VOLTA_SCRIPT)

if p.results:
    output = p.results[0].stdout.strip()
    print(output)

print("✅ Volta Scheduler a été complètement supprimé du cluster !")

# Libération des Ressources

In [ ]:
# Destruction de la réservation
print("Libération des ressources sur Grid'5000...")
provider.destroy()
print("Ressources libérées. ✅")

## 🔥 Script Stress-NG FINAL avec votre commande testée

In [ ]:
# Script stress-ng SIMPLIFIÉ - juste le stress sans monitoring
SIMPLE_STRESS_SCRIPT = """
#!/bin/bash

# Installation de stress-ng si nécessaire
if ! command -v stress-ng &> /dev/null; then
    sudo apt-get update -qq
    sudo apt-get install -y stress-ng
fi

# Lancement du stress
stress-ng --cpu 52 \\
          --cpu-method all \\
          --vm 8 \\
          --vm-bytes 80% \\
          --vm-method all \\
          --io 16 \\
          --hdd 8 \\
          --cache 52 \\
          --cache-level 3 \\
          --matrix 26 \\
          --vecmath 26 \\
          --timeout 600s \\
          --metrics-brief
"""

print("🔥 Lancement du stress-ng simplifié...")
print("⏱️  Durée: 5 minutes")
print("")

# Lancement sur tous les workers
with en.actions(roles=worker_nodes) as p:
    p.shell(SIMPLE_STRESS_SCRIPT)

if p.results:
    for i, result in enumerate(p.results):
        print(f"WORKER {i+1} ({result.host}): ", end="")
        if result.status == "OK":
            print("✅ Stress terminé")
        else:
            print(f"❌ Erreur: {result.stderr}")

## 🔍 Examen des Limites de Puissance RAPL

In [ ]:
# Script pour examiner les limites de puissance RAPL
RAPL_LIMITS_SCRIPT = """#!/bin/bash

echo "========================================================================"
echo "🔍 EXAMEN DES LIMITES DE PUISSANCE RAPL"
echo "========================================================================"
echo "Hostname: $(hostname)"
echo "Date: $(date)"
echo ""

# Vérification de l'existence du répertoire RAPL
RAPL_DIR="/sys/devices/virtual/powercap/intel-rapl"

if [ ! -d "$RAPL_DIR" ]; then
    echo "❌ Répertoire RAPL non trouvé: $RAPL_DIR"
    echo "Intel RAPL non disponible sur ce système"
    exit 1
fi

echo "✅ Répertoire RAPL trouvé: $RAPL_DIR"
echo ""

echo "--- Structure du répertoire RAPL ---"
ls -la $RAPL_DIR/
echo ""

echo "--- Recherche des fichiers *_power_limit_uw ---"
POWER_LIMIT_FILES=$(find $RAPL_DIR -name "*_power_limit_uw" 2>/dev/null)

if [ -z "$POWER_LIMIT_FILES" ]; then
    echo "❌ Aucun fichier *_power_limit_uw trouvé"
else
    echo "📊 Fichiers de limites de puissance trouvés:"
    echo "$POWER_LIMIT_FILES"
    echo ""
    
    echo "========================================================================"
    echo "📈 VALEURS DES LIMITES DE PUISSANCE"
    echo "========================================================================"
    
    for file in $POWER_LIMIT_FILES; do
        echo ""
        echo "--- Fichier: $file ---"
        
        if [ -r "$file" ]; then
            VALUE_UW=$(cat "$file" 2>/dev/null)
            if [ -n "$VALUE_UW" ] && [ "$VALUE_UW" -ne 0 ] 2>/dev/null; then
                # Conversion en Watts
                VALUE_W=$(echo "scale=2; $VALUE_UW / 1000000" | bc 2>/dev/null || echo "N/A")
                echo "  Valeur brute (microWatts): $VALUE_UW"
                echo "  Valeur en Watts: $VALUE_W W"
                
                # Déterminer le type de domaine
                DOMAIN_NAME=$(basename $(dirname "$file"))
                echo "  Domaine: $DOMAIN_NAME"
                
                # Afficher le nom du domaine si disponible
                NAME_FILE="$(dirname "$file")/name"
                if [ -r "$NAME_FILE" ]; then
                    DOMAIN_FRIENDLY_NAME=$(cat "$NAME_FILE" 2>/dev/null)
                    echo "  Nom du domaine: $DOMAIN_FRIENDLY_NAME"
                fi
                
            else
                echo "  ⚠️  Valeur non disponible ou nulle"
            fi
        else
            echo "  ❌ Fichier non accessible en lecture"
        fi
    done
fi

echo ""
echo "--- Informations additionnelles RAPL ---"
echo ""

# Afficher tous les domaines RAPL
echo "📋 Tous les domaines RAPL disponibles:"
for domain_dir in $RAPL_DIR/intel-rapl:*; do
    if [ -d "$domain_dir" ]; then
        DOMAIN_NAME=$(basename "$domain_dir")
        NAME_FILE="$domain_dir/name"
        if [ -r "$NAME_FILE" ]; then
            FRIENDLY_NAME=$(cat "$NAME_FILE" 2>/dev/null)
            echo "  $DOMAIN_NAME: $FRIENDLY_NAME"
            
            # Afficher la limite maximale si disponible
            MAX_LIMIT_FILE="$domain_dir/max_power_range_uw"
            if [ -r "$MAX_LIMIT_FILE" ]; then
                MAX_VALUE_UW=$(cat "$MAX_LIMIT_FILE" 2>/dev/null)
                if [ -n "$MAX_VALUE_UW" ]; then
                    MAX_VALUE_W=$(echo "scale=2; $MAX_VALUE_UW / 1000000" | bc 2>/dev/null || echo "N/A")
                    echo "    Limite maximale: $MAX_VALUE_W W"
                fi
            fi
        fi
    fi
done

echo ""
echo "--- Vérification des permissions ---"
echo "Permissions sur le répertoire RAPL:"
ls -ld $RAPL_DIR
echo ""
echo "Permissions sur les fichiers de limites:"
for file in $POWER_LIMIT_FILES; do
    echo "$(ls -l "$file")"
done

echo ""
echo "========================================================================"
echo "✅ Examen des limites RAPL terminé"
echo "========================================================================"
"""

print("🔍 Examen des limites de puissance RAPL sur les nœuds WORKERS uniquement...")
print("📊 Recherche des fichiers *_power_limit_uw dans /sys/devices/virtual/powercap/intel-rapl")
print("")

# Créer le script sur les workers uniquement et l'exécuter
with en.actions(roles=worker_nodes) as p:
    # Créer le fichier script
    p.copy(content=RAPL_LIMITS_SCRIPT, dest="/tmp/check_rapl_limits.sh", mode="0755")

with en.actions(roles=worker_nodes) as p:
    # Exécuter le script
    p.shell("bash /tmp/check_rapl_limits.sh")

if p.results:
    for i, result in enumerate(p.results):
        print(f"\n{'='*60}")
        print(f"📍 WORKER {i+1}: {result.host}")
        print('='*60)
        
        if result.status == "OK":
            # Extraire les informations importantes
            lines = result.stdout.split('\n')
            
            # Afficher les limites de puissance trouvées
            in_power_section = False
            for line in lines:
                if "VALEURS DES LIMITES DE PUISSANCE" in line:
                    in_power_section = True
                    print("📈 LIMITES DE PUISSANCE:")
                elif "Examen des limites RAPL terminé" in line:
                    in_power_section = False
                elif in_power_section and ("Valeur en Watts:" in line or "Domaine:" in line or "Nom du domaine:" in line):
                    print(f"  {line.strip()}")
                elif "Aucun fichier *_power_limit_uw trouvé" in line:
                    print("❌ Aucune limite de puissance RAPL trouvée")
                elif "Répertoire RAPL non trouvé" in line:
                    print("❌ Intel RAPL non disponible")
            
            # Afficher les domaines disponibles
            print("\n📋 Domaines RAPL:")
            for line in lines:
                if ": " in line and ("package" in line.lower() or "core" in line.lower() or "dram" in line.lower()):
                    print(f"  {line.strip()}")
                    
        else:
            print(f"❌ Erreur: {result.stderr}")

print(f"\n{'='*60}")
print("💡 INFO: Les limites RAPL permettent de contrôler la consommation maximale")
print("📊 Ces valeurs sont utilisées par Kepler pour le monitoring énergétique")
print("🖥️  Analyse effectuée uniquement sur les nœuds workers")
print('='*60)